# Decorators

Python decorators are a powerful concept that allow you to "wrap" a function with another function. A function can be decorated many times, if desired.

* [functools](https://docs.python.org/3/library/functools.html) — Higher-order functions and operations on callable objects
* [How to use functools.wraps](https://www.blog.pythonlibrary.org/2016/02/17/python-functools-wraps/)

## Functions That Return Other Functions

When you define a function in python, that function becomes an object.

In [1]:
def apply(func, arg):
    return func(arg)

In [2]:
apply(len, "hello")

5

In [7]:
def add(num):
    def adder(x):
        return num + x
    return adder

In [8]:
func = add(10)

In [9]:
func(5)

15

In [10]:
add(1)(10)

11

In [11]:
def string_required(f):
    def validate(arg):
        if type(arg) != str:
            raise ValueError(f'{repr(arg)} is not a string')
        return f(arg)
    return validate

In [12]:
string_required(len)("limited len")

11

In [13]:
string_required(len)([0, 1, 2, 3])

ValueError: [0, 1, 2, 3] is not a string

## Decorators Are A Syntax Sugar

In [15]:
@string_required
def greeting(name):
    print(f'Hi, {name}!')

In [16]:
greeting('Andrey')

Hi, Andrey!


In [17]:
greeting(1)

ValueError: 1 is not a string

In [18]:
@string_required
class Greeter:
    def __init__(self, name):
        self.name = name
        
    def say(self):
        print(f'Hi, {self.name}!')

In [19]:
Greeter('Andrey').say()

Hi, Andrey!


In [20]:
Greeter(('Andrey', 'Maria', 'Mark')).say()

ValueError: ('Andrey', 'Maria', 'Mark') is not a string

## Injecting Logic Into Class Methods

In [21]:
def barking(cls):
    for name in cls.__dict__:
        if name.startswith('__'):
            continue
        
        func = getattr(cls, name)

        def woofer(*args, **kwargs):
            print('Woof')
            return func(*args, **kwargs)

        setattr(cls, name, woofer)
        
    return cls

In [22]:
@barking
class Dog1:
    def shout(self):
        print("I'm a dog!")

In [23]:
d = Dog1()

In [24]:
d

In [27]:
d.shout()

Woof
I'm a dog!


## Decorators Can Return Something That Is Not A Function

The decorator has the ability to swallow the function, or return something that is not a function, if it wanted.

In [28]:
def stupid(cls):
    class Null:
        pass
    return Null

In [29]:
@stupid
class Something:
    def method1(self):
        pass
    def method2(self):
        pass

In [30]:
Something

__main__.stupid.<locals>.Null

In [31]:
Something.method1()

AttributeError: type object 'Null' has no attribute 'method1'